This script is to understand the word embedding and how it works.

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import torch.nn.functional as f

In [3]:
checkpoint = "google/gemma-2b"

In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint
)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [6]:
vocab = tokenizer.get_vocab()
print("Vocabulary size:", len(vocab))

Vocabulary size: 256000


In [12]:
text1 = "King"
text2 = "Queen"
inputs1 = tokenizer(text1, return_tensors="pt", padding=True, truncation=True)
inputs2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)

# Get embeddings from the model
with torch.no_grad():
    outputs1 = model(**inputs1, output_hidden_states=True)
    outputs2 = model(**inputs2, output_hidden_states=True)

# Use CLS token embeddings (first token in the sequence)
embedding1 = outputs1.last_hidden_state[:, 0, :]  # CLS token
embedding2 = outputs2.last_hidden_state[:, 0, :]


AttributeError: 'CausalLMOutputWithPast' object has no attribute 'last_hidden_state'

In [13]:
print(outputs1.last_hidden_state)

AttributeError: 'CausalLMOutputWithPast' object has no attribute 'last_hidden_state'

In [ ]:
# Normalize embeddings
embedding1 = f.normalize(embedding1, p=2, dim=1)
embedding2 = f.normalize(embedding2, p=2, dim=1)

# Compute cosine similarity
cosine_sim = torch.mm(embedding1, embedding2.T)
print(f"Cosine similarity: {cosine_sim.item()}")